# Practice using pykep's `direct_pl2pl` UDP and pygmo

pykep doc: https://esa.github.io/pykep/documentation/trajopt.html#pykep.trajopt.direct_pl2pl

Yuri Shimane, 2022.04.21

In [92]:
import numpy as np
import pygmo as pg
import pykep as pk

%matplotlib notebook
%load_ext autoreload
%autoreload 2

### Launch epoch

First, a word on launch epoch. Pykep's UDPs usually want this in mjd2000. So we first create `pk.epoch` objects, which we then convert to floats. 

In [56]:
epoch1 = pk.epoch_from_string("2030-01-01 12:23:00.000")
epoch2 = pk.epoch_from_string("2032-01-01 12:23:00.000")
epoch1.mjd2000

10958.515972222222

In [57]:
# we construct our problem
prob = pk.trajopt.direct_pl2pl(
    p0 = "earth",
    pf = "mars",
    mass = 2000,   # kg
    thrust = 0.4,  # thrust in N
    isp = 3000,    # isp in seconds
    nseg = 30,     # number of segments in Sims-Flanagan transcription
    t0 = [epoch1.mjd2000, epoch2.mjd2000],  # Launch epochs bounds [mjd2000]
    tof = [150, 500],  # 
    vinf_dep = 2.0,   # Launch DV, km/s
    vinf_arr = 0.0,   # Arrival DV, km/s
)

In [58]:
class add_gradient:
    """Construct pygmo UDP with gradient
    https://esa.github.io/pygmo2/tutorials/coding_udp_constrained.html?highlight=add_gradient
    """
    def __init__(self, prob):
        self.prob = pg.problem(prob)

    def fitness(self, x):
        return self.prob.fitness(x)

    def get_bounds(self):
        return self.prob.get_bounds()

    def get_nec(self):
        return self.prob.get_nec()

    def get_nic(self):
        return self.prob.get_nic()

    def get_nobj(self):
        return self.prob.get_nobj()
        
    def gradient(self, x):
        return pg.estimate_gradient(lambda x: self.fitness(x), x) # we here use the low precision gradient

In [59]:
prob_with_gradient = add_gradient(prob)

In [96]:
# create ipopt algorithm class
uda = pg.ipopt() # pylint: disable=no-member
uda.set_integer_option("print_level", 5)
uda.set_integer_option("acceptable_iter", 4)
uda.set_integer_option("max_iter", 5000)      # probably need thousands of iterations

uda.set_numeric_option("tol", 1e-5)
uda.set_numeric_option("dual_inf_tol", 1e-6)
uda.set_numeric_option("constr_viol_tol", 1e-5)
uda.set_numeric_option("compl_inf_tol", 1e-6)

uda.set_numeric_option("acceptable_tol", 1e-3)
uda.set_numeric_option("acceptable_dual_inf_tol", 1e-2)
uda.set_numeric_option("acceptable_constr_viol_tol", 1e-5)
uda.set_numeric_option("acceptable_compl_inf_tol", 1e-5)

algo = pg.algorithm(uda)

In [97]:
algo

Algorithm name: Ipopt: Interior Point Optimization [deterministic]
	C++ class name: class pagmo::ipopt

	Thread safety: none

Extra info:
	Last optimisation return code: Solve_Succeeded (value = 0)
	Verbosity: 0
	Individual selection policy: best
	Individual replacement policy: best
	Integer options: {acceptable_iter : 4,  max_iter : 5000,  print_level : 5}
	Numeric options: {acceptable_compl_inf_tol : 1e-05,  acceptable_constr_viol_tol : 1e-05,  acceptable_dual_inf_tol : 0.01,  acceptable_tol : 0.001,  compl_inf_tol : 1e-06,  ...}

# Create a population

We now create a population. Here, we are not trying to do a thorough global search, so we initialize the population with `size=1`. This is equivalent to using the algorithm with a randomized initial guess, once. 
If we want to use an initial guess that we have a priori (say `x0`), then we should initialize the population with `size=0` then push back:

```python
# ... assume we have some initial guess vector x0 that we'd like to use
pop = pg.population(prob_with_gradient, size=0)   # initialize with size=0
pop.push_back(xopt)                               # now size is +1
```

In [98]:
pop = pg.population(prob_with_gradient, size=1)
pop

Problem name: <class '__main__.add_gradient'>
	C++ class name: class pybind11::object

	Global dimension:			99
	Integer dimension:			0
	Fitness dimension:			40
	Number of objectives:			1
	Equality constraints dimension:		7
	Inequality constraints dimension:	32
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [10958.5, 150, 200, -2000, -2000, ... ]
	Upper bounds: [11688.5, 500, 2000, 2000, 2000, ... ]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 3960
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 1
	Gradient evaluations: 0

	Thread safety: none

Population size: 1

List of individuals: 
#0:
	ID:			12692838010386895863
	Decision vector:	[11246.6, 169.257, 1190.45, 1782.38, 562.858, ... ]
	Fitness vector:		[-1190.45, 1.37493, 0.758595, -0.104617, -0.825239, ... ]

Champion decision vector: [11246.6, 169.257, 1190.45, 1782.38, 562.858, ... ]
Champion fitness:

**CAVEAT**: when running with Jupyter notebooks, the algorithm iteration messages are shown on the shell session running Jupyter notebook, not on the notebook `Out` cell itself...

In [ ]:
# solving with ipopt
pop = algo.evolve(pop)

In [ ]:
algo

### Visualize result

See also: https://esa.github.io/pykep/documentation/trajopt.html#pykep.trajopt.direct_pl2pl

In [ ]:
xopt, fopt = pop.champion_x, pop.champion_f

In [ ]:
pop.champion_f

In [ ]:
prob.pretty(xopt)

In [ ]:
prob.plot_traj(xopt)

In [ ]:
prob.plot_control(xopt)